# Conjuguaison de verbes en francais

## Preparation de donnnée

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

2024-03-13 04:20:08.579501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Version 1

In [ ]:
# Exemple de structure de votre DataFrame
df = pd.DataFrame({
    'verbe_infinitif': ['prendre', 'prendre', ...],
    'temps_verbal': ['présent de l\'indicatif', 'présent du subjonctif', ...],
    'personne': ['première singulier', 'première singulier', ...],
    'verbe_conjugue': ['je prends', 'que je prenne', ...]
})

# Supposons df est votre DataFrame après avoir chargé et préparé R

# Séparation en ensembles d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.2)

# Tokenization et conversion en séquences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([train_df['verbe_infinitif'], train_df['temps_verbal'], train_df['personne'], train_df['verbe_conjugue']]))
vocab_size = len(tokenizer.word_index) + 1

# Préparation des séquences d'entrée et de sortie
encoder_input_data = tokenizer.texts_to_sequences(train_df[['verbe_infinitif', 'temps_verbal', 'personne']].apply(lambda x: ' '.join(x), axis=1))
decoder_input_data = tokenizer.texts_to_sequences(train_df['verbe_conjugue'])
decoder_target_data = pad_sequences(decoder_input_data, maxlen=None, padding='post', truncating='post')

encoder_input_data = pad_sequences(encoder_input_data, maxlen=max(encoder_input_data.keymap(len)), padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max(decoder_input_data.keymap(len)) - 1, padding='post')  # -1 pour shift
decoder_target_data = to_categorical(decoder_target_data, num_classes=vocab_size)

# Note: Vous devez également préparer les données de test de manière similaire pour l'évaluation


### Version 2

1. Chargement de la base de données

In [ ]:
data = pd.read_csv('liste')

In [6]:
# Simulons le chargement de votre DataFrame R
# Pour cet exemple, nous utilisons des listes simplifiées
data = {
    'verbe_infinitif': ['prendre', 'prendre', 'prendre'],
    'temps_verbal': ['présent de l\'indicatif', 'présent du subjonctif', 'passé simple'],
    'personne': ['première singulier', 'première singulier', 'première pluriel'],
    'verbe_conjugue': ['je prends', 'que je prenne', 'nous prîmes']
}

# Convertissons notre structure de données en DataFrame pour une manipulation facile
df = pd.DataFrame(data)

# Fusionnons les colonnes d'entrée en une seule chaîne pour simplifier la tokenisation
df['input'] = df[['verbe_infinitif', 'temps_verbal', 'personne']].agg(' '.join, axis=1)

# Tokenisons les entrées et les sorties
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([df['input'], df['verbe_conjugue']]))
vocab_size = len(tokenizer.word_index) + 1

# Convertissons les textes en séquences d'entiers
sequence_inputs = tokenizer.texts_to_sequences(df['input'])
sequence_outputs = tokenizer.texts_to_sequences(df['verbe_conjugue'])

# Paddons les séquences pour qu'elles aient toutes la même longueur
max_seq_length = max(max(len(seq) for seq in sequence_inputs), max(len(seq) for seq in sequence_outputs))
encoder_input_data = pad_sequences(sequence_inputs, maxlen=max_seq_length, padding='post')
decoder_input_data = pad_sequences(sequence_outputs, maxlen=max_seq_length, padding='post')

# Préparons les données de sortie (décalées d'un pas de temps et catégorisées)
decoder_output_data = np.zeros_like(decoder_input_data)
decoder_output_data[:, 0:-1] = decoder_input_data[:, 1:]
decoder_output_data = to_categorical(decoder_output_data, num_classes=vocab_size)

# Séparation des données en ensembles d'entraînement et de test
encoder_input_train, encoder_input_test, decoder_input_train, decoder_input_test, decoder_output_train, decoder_output_test = train_test_split(encoder_input_data, decoder_input_data, decoder_output_data, test_size=0.2, random_state=42)


## Modele encodeur decodeur

In [2]:
import tensorflow as tf

from tensorflow.keras.metrics import Metric

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, TimeDistributed, Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

2024-03-12 12:37:36.605171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
class ExactMatch(Metric):
    def __init__(self, name='exact_match', **kwargs):
        super(ExactMatch, self).__init__(name=name, **kwargs)
        self.correct_predictions = self.add_weight(name='cp', initializer='zeros')
        self.total_predictions = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        matches = tf.cast(tf.equal(tf.argmax(y_true, axis=-1), y_pred), 'float32')
        self.correct_predictions.assign_add(tf.reduce_sum(matches))
        self.total_predictions.assign_add(tf.size(matches))

    def result(self):
        return self.correct_predictions / self.total_predictions

    def reset_states(self):
        self.correct_predictions.assign(0.)
        self.total_predictions.assign(0.)

### Sans mecanisme d'attention

In [ ]:
# Définition des dimensions
embedding_dim = 256
lstm_units = 512
vocab_size = len(tokenizer.word_index) + 1  # Assurez-vous que tokenizer a été défini lors de la préparation des données

# Encodeur
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Décodeur
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Modèle
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model.summary()

In [ ]:
# Entraînement du modèle
model.fit([encoder_input_train, decoder_input_train], decoder_output_train,
          batch_size=64,
          epochs=100,
          validation_split=0.2)

In [ ]:
# Évaluation du modèle sur les données de test
model.evaluate([encoder_input_test, decoder_input_test], decoder_output_test)

### Avec mecanisme d'attention

In [ ]:
# Utilisation des valeurs définies lors de la préparation des données
vocab_size = len(tokenizer.word_index) + 1  # Ajoutez 1 pour le token 0 qui n'est pas utilisé
max_seq_length = max_seq_length  # Défini lors de la préparation des données

# Hypothétiques dimensions d'embedding et unités LSTM
embedding_dim = 256
lstm_units = 512

# Encodeur
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Décodeur
decoder_inputs = Input(shape=(max_seq_length,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Mécanisme d'attention
attention_layer = Attention()
attention_result = attention_layer([decoder_outputs, encoder_outputs])
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attention_result])

# Pour la génération de la sortie
decoder_dense = TimeDistributed(Dense(vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Construction du modèle
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model.summary()


In [ ]:
# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit([encoder_input_train, decoder_input_train], decoder_output_train,
          batch_size=64,
          epochs=100,
          validation_split=0.2)


In [ ]:
# Évaluation du modèle sur les données de test
model.evaluate([encoder_input_test, decoder_input_test], decoder_output_test)